In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# อ่านไฟล์ข้อมูล
non_phishing_df = pd.read_csv('Non-Phishing.csv')
phishing_df = pd.read_csv('Phishing.csv')

In [4]:
# จัดการคอลัมน์ให้สอดคล้องกัน
# เรียงคอลัมน์ให้เหมือนกัน: sender, receiver, date, subject, body, urls, label
non_phishing_df = non_phishing_df[['sender', 'receiver', 'date', 'subject', 'body', 'urls', 'label']]
phishing_df = phishing_df[['sender', 'receiver', 'date', 'subject', 'body', 'urls', 'label']]

In [5]:
# ทำความสะอาดข้อมูล โดยลบแถวที่มีค่าว่างในคอลัมน์ที่สำคัญ
all_emails = pd.concat([non_phishing_df, phishing_df], ignore_index=True)
all_emails.dropna(subset=['sender', 'receiver', 'date', 'subject', 'body', 'urls', 'label'], inplace=True)

In [6]:
# แบ่งข้อมูลเป็นฟีเจอร์และเลเบล
X = all_emails.drop('label', axis=1)
y = all_emails['label']

In [7]:
# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ โดยให้มีอัตราส่วนของอีเมลธรรมดาและอีเมล phishing ใกล้เคียงกัน
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
# บันทึกชุดข้อมูลฝึกและทดสอบลงไฟล์
X_train.to_csv('Training_Set.csv', index=False)
y_train.to_csv('Training_Labels.csv', index=False)
X_test.to_csv('Test_Set.csv', index=False)
y_test.to_csv('Test_Labels.csv', index=False)